In [ ]:
import pandas as pd

df=pd.read_pickle("../data/시도_승합_승용.pkl")

In [6]:
df.columns

Index(['sido_id', 'reg_year', 'vehicle_type', 'car_count'], dtype='object')

In [9]:
df.vehicle_type.unique()

array(['car', 'van'], dtype=object)

In [10]:
df.reg_year.unique()

array([2022, 2023, 2024])

In [13]:
SIDO_MAP = {
    "01": "서울특별시",
    "02": "강원특별자치도",
    "03": "대전광역시",
    "04": "충청북도",
    "05": "충청남도",
    "06": "인천광역시",
    "07": "경기도",
    "08": "광주광역시",
    "10": "전라남도",
    "11": "부산광역시",
    "12": "울산광역시",
    "13": "경상남도",
    "14": "제주특별자치도",
    "15": "대구광역시",
    "16": "경상북도",
    "18": "전북특별자치도",
    "36": "세종특별자치시",
}


In [ ]:
import pandas as pd

def build_sido_map_df(df_all: pd.DataFrame, vehicle_type: str) -> pd.DataFrame:
    """
    input: columns = ['sido_id','reg_year','vehicle_type','car_count']
    output: columns = ['sido_name','2022.12 월','2023.12 월','2024.12 월', ...]
    """
    df = df_all.copy()

    # 1) sido_id 정리 + sido_name 매핑
    df["sido_id"] = df["sido_id"].astype(str).str.zfill(2)
    df["sido_name"] = df["sido_id"].map(SIDO_MAP)

    # 매핑 실패(누락) 방지: 이름 없으면 코드 그대로라도 남김
    df["sido_name"] = df["sido_name"].fillna(df["sido_id"])

    # 2) 차종 필터 (vehicle_type: 'car' or 'van')
    df = df[df["vehicle_type"] == vehicle_type].copy()

    # 3) 피벗: sido_name x 연도
    wide = (
        df.pivot_table(index="sido_name", columns="reg_year", values="car_count", aggfunc="sum")
          .reset_index()
    )

    # 4) make_sido_*_map이 찾는 컬럼명 형식으로 변경
    wide.columns = ["sido_name"] + [f"{int(y)}.12 월" for y in wide.columns[1:]]

    return wide


In [ ]:
import pandas as pd

PKL_PATH = "../data/시도_승합_승용.pkl"   # 스트림릿 프로젝트 경로

df_all = pd.read_pickle(PKL_PATH)

# ✅ 지도용 DF 만들기 (와이드폼)
sidocar_df = build_sido_map_df(df_all, "car")
sidovan_df = build_sido_map_df(df_all, "van")

# ✅ 연도별 지도 생성
maps_car = {}
maps_van = {}

for year in [2022, 2023, 2024]:
    maps_car[year] = make_sido_car_map(sidocar_df, year)
    maps_van[year] = make_sido_van_map(sidovan_df, year)
